# Set up (+ reproducible results)

In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'

In [3]:
# Setting the seed for numpy-generated random numbers
np.random.seed(54)

In [4]:
# Setting the seed for Python random numbers
rn.seed(28)

In [5]:
tf.set_random_seed(753)

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [7]:
train_path = 'towels/train/'
valid_path = 'towels/validate/'
test_path = 'towels/test/'

In [8]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['True', 'False'], batch_size=10) 
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['True', 'False'], batch_size=4) 
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['True', 'False'], batch_size=28) 

Found 239 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [9]:
imgs, labels = next(train_batches)

In [10]:
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

# Build Fine-tuned VGG16 model

In [41]:
inceptionv3_model = keras.applications.inception_v3.InceptionV3()

In [28]:
inceptionv3_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, None, None, 3 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, None, None, 3 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [42]:
type(inceptionv3_model)

keras.engine.training.Model

In [47]:
print(type(inceptionv3_model.layers[-2]))

<class 'keras.layers.pooling.GlobalAveragePooling2D'>


In [48]:
model = Sequential()
for layer in inceptionv3_model.layers[:-2]:
    #print(type(layer))
    model.add(layer)

ValueError: Input 0 is incompatible with layer conv2d_289: expected axis -1 of input shape to have value 192 but got shape (None, None, None, 64)

# Train the fine-tuned InceptionV3 model

# Predict using fine-tuned InceptionV3 model